In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization, Input, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ----------------- 1️⃣ LOAD DATA -----------------
def load_data(folder, label):
    data, labels = [], []
    for file in os.listdir(folder):
        if file.endswith(".xlsx"):
            df = pd.read_excel(os.path.join(folder, file))
            data.append(df.values)
            labels.append(label)
    return data, labels

control_folder = "F:\juypter notes\Gait assessment in Parkinson Desease\Raw Data in Excel - ID\Control Subjects\Arms"
patient_folder = "F:\juypter notes\Gait assessment in Parkinson Desease\Raw Data in Excel - ID\Patients\Arms"

control_data, control_labels = load_data(control_folder, label=0)
patient_data, patient_labels = load_data(patient_folder, label=1)

X = control_data + patient_data
y = np.array(control_labels + patient_labels)

# ----------------- 2️⃣ PREPROCESSING -----------------
max_len = min([len(seq) for seq in X])  # Instead of padding to max, truncate to min
X_truncated = [seq[:max_len] for seq in X]

scaler = StandardScaler()
X_scaled = np.array([scaler.fit_transform(seq) for seq in X_truncated])

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# ----------------- 3️⃣ CNN + LSTM MODEL -----------------
def create_cnn_lstm_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(64, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Conv1D(128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        LSTM(64, return_sequences=True, recurrent_dropout=0.2),
        LSTM(32, recurrent_dropout=0.2),
        Dropout(0.3),
        Flatten(),
        Dense(16, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, clipnorm=1.0), 
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_cnn_lstm_model((max_len, X_scaled.shape[2]))

early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4)

history = model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.2,
                    callbacks=[early_stopping, lr_scheduler])

# ----------------- 4️⃣ EVALUATE MODEL -----------------
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

y_pred = (model.predict(X_test) > 0.5).astype("int32")

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"Precision: {precision_score(y_test, y_pred):.2f}")
print(f"Recall: {recall_score(y_test, y_pred):.2f}")
print(f"F1-score: {f1_score(y_test, y_pred):.2f}")


Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.5208 - loss: 0.6553 - val_accuracy: 0.5000 - val_loss: 0.7037 - learning_rate: 0.0010
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 371ms/step - accuracy: 0.6042 - loss: 0.6917 - val_accuracy: 0.5000 - val_loss: 0.6984 - learning_rate: 0.0010
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 363ms/step - accuracy: 0.6875 - loss: 0.6855 - val_accuracy: 0.5000 - val_loss: 0.6920 - learning_rate: 0.0010
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 383ms/step - accuracy: 0.6875 - loss: 0.5797 - val_accuracy: 0.5000 - val_loss: 0.6928 - learning_rate: 0.0010
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 402ms/step - accuracy: 0.7917 - loss: 0.5889 - val_accuracy: 0.5000 - val_loss: 0.7081 - learning_rate: 0.0010
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 377ms/step - accuracy: 0.6875 - loss: 0.5497 - val_accuracy: 0.5000 - val_loss: 0.7348 - learning_rate: 0.0010
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 385ms/step - accuracy: 0.6875 - loss: 0.5718 - val_accuracy: 0.5